In [2]:
import pandas as pd
import boto3
import os
s3 = boto3.session.Session().client(
            service_name="s3",
            endpoint_url=os.getenv("AWS_ENDPOINT_URL"),
            aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
            aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        )

In [3]:
s3.get('data-ice-lake-04', 'messager-data/analytics/geo-events/date=2022-01-01')

AttributeError: 'S3' object has no attribute 'ObjectSummary'